In [59]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [60]:
# Import dataset
print("Loading dataset...")
dataset = pd.read_csv(r"G:\Mon Drive\Fichiers\2.Scolarité\1. Jedha_Data_Science\CERTIF_PROJECTS\ML_Engineer_Certification_Projects\04_SUPERVISED_ML\Walmart\Src\Walmart_Store_sales.csv")
print("...Done.")
dataset.head()

Loading dataset...
...Done.


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [61]:
# Basic stats
print("general info : ")
display(dataset.info())
print()

print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

general info : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


None


Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [62]:
# Drop rows where weekly sales is missing
mask_1 = ~dataset['Weekly_Sales'].isnull()
dataset = dataset.loc[mask_1, :]
print(dataset.shape[0])

136


In [63]:
# Create date column in date Year / month / day / day of the week
dataset['Year'] = pd.to_datetime(dataset['Date']).dt.year
dataset['Month'] = pd.to_datetime(dataset['Date']).dt.month
dataset['Day'] = pd.to_datetime(dataset['Date']).dt.day
dataset['Week_day'] = pd.to_datetime(dataset['Date']).dt.dayofweek


dataset = dataset.drop(columns='Date')

C:\Users\mathi\AppData\Local\Temp\ipykernel_12248\2596267548.py:2: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Users\mathi\AppData\Local\Temp\ipykernel_12248\2596267548.py:3: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Users\mathi\AppData\Local\Temp\ipykernel_12248\2596267548.py:4: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Users\mathi\AppData\Local\Temp\ipykernel_12248\2596267548.py:5: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [64]:
display(dataset.head())
print(dataset.shape[0])

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_day
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


136


In [65]:
display(100*dataset.isnull().sum()/dataset.shape[0])

Store            0.000000
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
Year            13.235294
Month           13.235294
Day             13.235294
Week_day        13.235294
dtype: float64

In [66]:
def drop_outliers (dataset, columns):
    for col in columns:
        mean = dataset[col].mean()
        std = dataset[col].std()
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
        filtered_df = dataset[dataset[col].between(lower_bound, upper_bound)]

    return filtered_df

In [67]:
outlier_columns = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

dataset_filtered = drop_outliers(dataset.copy(), outlier_columns)

In [68]:
display(dataset_filtered.head())
print(dataset_filtered.shape[0])

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_day
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


117


### Remove missing values in date. We will create a new dataset, and test later without
### this pre-processing to compare performances.

In [69]:
print((dataset_filtered["Year"].isnull()).value_counts())
dataset_time_notnull = dataset_filtered[dataset_filtered['Year'].notnull()]
print(dataset_time_notnull.shape[0])

Year
False    102
True      15
Name: count, dtype: int64
102


### Model_1 Training

In [70]:
target_variable = "Weekly_Sales"

X = dataset_time_notnull.drop(target_variable, axis = 1)
Y = dataset_time_notnull.loc[:,target_variable]

print(X.shape)
print(Y.shape)

(102, 10)
(102,)


In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [72]:
categorical_features = ['Store','Holiday_Flag']
numeric_features = list(set(X.columns) - set(categorical_features))


print(numeric_features)
print(categorical_features)

['Temperature', 'Fuel_Price', 'Year', 'Week_day', 'Unemployment', 'Day', 'CPI', 'Month']
['Store', 'Holiday_Flag']


In [73]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(drop="first")),
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



print("Performing preprocessings on train set...")
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

print("Performing preprocessings on test set...")
X_test = preprocessor.transform(X_test)
print('...Done.')
print(X_test[0:5,:])
print()


print("Reshaping target")
Y_train = Y_train.values.reshape(-1,1)
print("...Done")
print(Y_train[0:5])
print()
Y_test = Y_test.values.reshape(-1,1)
print("...Done")
print(Y_test[0:5])

Performing preprocessings on train set...
...Done.
[[-1.04457662 -0.70559168  0.13363062  0.          0.75510203 -1.07663405
   0.7460212  -1.69030851  1.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.08677032 -1.16940974 -1.06904497  0.          0.61324925  1.36843206
  -1.55554271 -0.38635623  0.          0.          1.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [-1.81641811  0.26210145  1.33630621  0.         -1.31250969  0.43697831
  -1.44388639 -1.69030851  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.90306678 -1

In [74]:
print("Train model...")
model_1 = LinearRegression()
model_1.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


### Model_1 Performance assessment

In [75]:
print("R2 score on training set : ", model_1.score(X_train, Y_train))
print("R2 score on test set : ", model_1.score(X_test, Y_test))

mse_model_1 = mean_squared_error(Y_test, model_1.predict(X_test))
print("Mean Squared Error:", mse_model_1)

R2 score on training set :  0.9859923021083215
R2 score on test set :  0.9104953289320066
Mean Squared Error: 43313516398.01707


### Extracting coefficients and ploting them

In [76]:
coefficients = model_1.coef_
print("Coefficients:", coefficients)

Coefficients: [[-3.41829710e+04 -6.42994768e+04  7.64900804e+03  5.60248736e-09
  -9.78420505e+04 -6.03129690e+04  1.10779562e+05  3.31811461e+04
   1.65433404e+05 -1.29781206e+06  6.77141738e+05 -1.43514100e+06
  -1.34000917e+05 -9.54805696e+05 -8.69505182e+05 -1.31533660e+06
   6.12443459e+05  1.90997466e+05  4.74588204e+05  6.70557689e+05
  -6.69478910e+05 -1.15194484e+06 -7.60066217e+05 -1.66055520e+05
   1.17227077e+05  3.59312624e+05 -9.12239666e+04]]


In [77]:
# Check
print((X_train).shape)
print((coefficients).shape)

(81, 27)
(1, 27)


In [78]:
encoded_feature_names = preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(categorical_features)

final_feature_names = numeric_features + list(encoded_feature_names)

In [79]:
# Check
print(len(final_feature_names))
final_feature_names

27


['Temperature',
 'Fuel_Price',
 'Year',
 'Week_day',
 'Unemployment',
 'Day',
 'CPI',
 'Month',
 'Store_2.0',
 'Store_3.0',
 'Store_4.0',
 'Store_5.0',
 'Store_6.0',
 'Store_7.0',
 'Store_8.0',
 'Store_9.0',
 'Store_10.0',
 'Store_11.0',
 'Store_13.0',
 'Store_14.0',
 'Store_15.0',
 'Store_16.0',
 'Store_17.0',
 'Store_18.0',
 'Store_19.0',
 'Store_20.0',
 'Holiday_Flag_1.0']

In [80]:
feature_coef = pd.DataFrame({'feature': final_feature_names, 'coefficient': coefficients.flatten()})
feature_coef.head()

,feature,coefficient
0,Temperature,-3.418297e+04
1,Fuel_Price,-6.429948e+04
2,Year,7.649008e+03
3,Week_day,5.602487e-09
4,Unemployment,-9.784205e+04


In [81]:
fig = px.bar(feature_coef, x="feature", y="coefficient",  title='Histogram of Coefficients')
fig.update_xaxes(title_text='Coefficient')
fig.show()

### Model_2 using GridSearchCV, trying both Lasso and Ridge regularizations

In [82]:
#Pipeline
ridge_pipeline = Pipeline([
    ('ridge', Ridge())
])

lasso_pipeline = Pipeline([
    ('lasso', Lasso())
])

#Parameters
ridge_params = {
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
}
lasso_params = {
    'lasso__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
}

#Training models
ridge_gridsearch = GridSearchCV(estimator = ridge_pipeline,
                                param_grid = ridge_params,
                                cv = 3)
ridge_gridsearch.fit(X_train, Y_train)

lasso_gridsearch = GridSearchCV(estimator = lasso_pipeline,
                                param_grid = lasso_params,
                                cv = 3)
lasso_gridsearch.fit(X_train, Y_train)

print("...Done.")

print("Best parameters for Ridge:", ridge_gridsearch.best_params_)
print("Best score for Ridge:", ridge_gridsearch.best_score_)

print("Best parameters for Lasso:", lasso_gridsearch.best_params_)
print("Best score for Lasso:", lasso_gridsearch.best_score_)

...Done.
Best parameters for Ridge: {'ridge__alpha': 0.01}
Best score for Ridge: 0.895404626246266
Best parameters for Lasso: {'lasso__alpha': 0.001}
Best score for Lasso: 0.8936141912396843


C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.625e+11, tolerance: 2.058e+09

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.620e+11, tolerance: 2.058e+09

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:678

In [83]:
# Training models 

ridge_model = Pipeline([
    ('ridge', Ridge(alpha=0.01))
])

lasso_model = Pipeline([
    ('lasso', Lasso(alpha=0.001))
])


#Training models
ridge_model.fit(X_train, Y_train)
lasso_model.fit(X_train, Y_train)

Pipeline(steps=[('lasso', Lasso(alpha=0.001))])

#### Ridge performance assessment

In [84]:
print("R2 score on training set : ", ridge_model.score(X_train, Y_train))
print("R2 score on test set : ", ridge_model.score(X_test, Y_test))

mse_ridge_model = mean_squared_error(Y_test, ridge_model.predict(X_test))
print("Mean Squared Error:", mse_ridge_model)

R2 score on training set :  0.985950103855604
R2 score on test set :  0.911507940670722
Mean Squared Error: 42823488619.28404


#### Ridge performance assessment

In [85]:
print("R2 score on training set : ", lasso_model.score(X_train, Y_train))
print("R2 score on test set : ", lasso_model.score(X_test, Y_test))

mse_lasso_model = mean_squared_error(Y_test, ridge_model.predict(X_test))
print("Mean Squared Error:", mse_lasso_model)

R2 score on training set :  0.9859923021083183
R2 score on test set :  0.910495336911868
Mean Squared Error: 42823488619.28404
